In [1]:
#!pip install guidance

In [2]:
import guidance

#from transformers import AutoModelForCausalLM
#model_name = "TheBloke/Wizard-Vicuna-13B-Uncensored-HF"
#model_8bit = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)

In [3]:
#!pip install sentencepiece
#!pip install tokenizers -U
#!pip install transformers[sentencepiece] -U
#!pip install accelerate -U
#!pip install optimum
#!pip install GPUtil
#https://huggingface.co/TheBloke/Wizard-Vicuna-30B-Uncensored-GPTQ/discussions/5


variable_name = "yhat_countries"
import pandas as pd
import os
max_new_tokens=200
temperature=0.7
import os

#https://stackoverflow.com/questions/73747731/runtimeerror-cuda-out-of-memory-how-setting-max-split-size-mb
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32" #setting this very small finally solved my OOM error

# Get the current working directory
cwd = os.getcwd()
cwd
os.chdir('/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/')
cwd = os.getcwd()
cwd
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1" #auto gets confused for some reason and puts it on the 2080
import torch
torch.backends.cudnn.benchmark = True #https://blog.paperspace.com/pytorch-memory-multi-gpu-debugging/
torch.backends.cudnn.enabled = True #https://blog.paperspace.com/pytorch-memory-multi-gpu-debugging/
torch.cuda.device_count()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.device_count()
torch.cuda.get_device_name(device=None)

import torch
torch.cuda.empty_cache()

import GPUtil
GPUtil.showUtilization()
import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

# If you've already downloaded the model, reference its location here:
#quantized_model_dir = "/path/to/TheBloke_WizardLM-Uncensored-Falcon-40B-3bit-GPTQ"
# Or to download it from the hub and store it in the Hugging Face cache directory:

#quantized_model_dir = "./data_temp/Wizard-Vicuna-13B-Uncensored-GPTQ/" #TheBloke_WizardLM-Uncensored-Falcon-40B-3bit-GPTQ
#quantized_model_dir = "TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ"
quantized_model_dir = "TheBloke/Wizard-Vicuna-30B-Uncensored-GPTQ"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(quantized_model_dir, use_fast=False)

#FileNotFoundError: Could not find model in TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ
#https://huggingface.co/TheBloke/wizard-vicuna-13B-GPTQ/discussions/7
#quantized_model_dir = "TheBloke/stable-vicuna-13B-GPTQ"
#model_basename = "wizard-vicuna-13B-GPTQ-4bit.compat.no-act-order" #YOU HAVE TO DO THIS

#model_basename = "Wizard-vicuna-30B-Uncensored-GPTQ-4bit.act-order"
# path to directory containing local model
#https://huggingface.co/TheBloke/starcoderplus-GPTQ/discussions/3
quantized_model_dir = "TheBloke/Wizard-Vicuna-30B-Uncensored-GPTQ" #"/workspace/models/TheBloke_Wizard-Vicuna-30B-Uncensored-GPTQ"
model_basename = "Wizard-Vicuna-30B-Uncensored-GPTQ-4bit.act.order" #.safetensors
use_triton = False 

#https://github.com/PanQiWei/AutoGPTQ
model = AutoGPTQForCausalLM.from_quantized(quantized_model_dir, 
                                           device="cuda:0", 
                                           use_triton=False,
                                           use_safetensors=True , 
                                           model_basename=model_basename,
                                           quantize_config=None,
                                           #torch_dtype=torch.bfloat16, 
                                           trust_remote_code=True
                                           ) #





| ID | GPU | MEM |
------------------
|  0 | 31% |  8% |
|  1 |  0% |  0% |


/home/skynet3/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-09 20:46:10,960 - 140010370217216 - modeling.py-modeling:1035 - WARNING: The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
2023-07-09 20:46:10,967 - 140010370217216 - modeling.py-modeling:928 - WARNING: The safetensors archive passed at /home/skynet3/.cache/huggingface/hub/models--TheBloke--Wizard-Vicuna-30B-Uncensored-GPTQ/snapshots/7aeb8db6d49732afa76dc7ce60c5c040dbbcf3ea/Wizard-Vicuna-30B-Uncensored-GPTQ-4bit.act.order.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.
2023-07-09 20:46:16,605 - 140010370217216 - fused_llama_mlp.py-fused_llama_mlp:306 - WARNING: skip module injec

In [4]:

#On how to do 4 bit, load the model like you would normally
#https://github.com/microsoft/guidance/issues/43
#https://github.com/microsoft/guidance/discussions/110
llama = guidance.llms.Transformers(model=model, tokenizer=tokenizer, device=0)
guidance.llm = llama


In [22]:



# we use LLaMA here, but any GPT-style model will do
#llama = guidance.llms.Transformers("TheBloke/Wizard-Vicuna-13B-Uncensored-HF", device=0)


# we can pre-define valid option sets
valid_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]

# define the prompt
character_maker = guidance("""The following is a character profile for an RPG game in JSON format.
```json
{
    "id": "{{id}}",
    "description": "{{description}}",
    "name": "{{gen 'name'}}",
    "age": {{gen 'age' pattern='[0-9]+' stop=','}},
    "armor": "{{#select 'armor'}}leather{{or}}chainmail{{or}}plate{{/select}}",
    "weapon": "{{select 'weapon' options=valid_weapons}}",
    "class": "{{gen 'class'}}",
    "mantra": "{{gen 'mantra' temperature=0.7}}",
    "strength": {{gen 'strength' pattern='[0-9]+' stop=','}},
    "items": [{{#geneach 'items' num_iterations=5 join=', '}}"{{gen 'this' temperature=0.7}}"{{/geneach}}]
}```""")

# generate a character
character_maker(
    id="e1f491f7-7ab8-4dac-8c20-c92b5e7d883d",
    description="A quick and nimble fighter.",
    valid_weapons=valid_weapons, llm=llama
)

Stop program The following is a character profile for an RPG game in JSON format.
```json
{
 "id": " {{id}} ",
 "description": " {{description}} ",
 "name": " {{gen 'name'}} ",
 "age": {{gen 'age' pattern='[0-9]+' stop=','}} ,
 "armor": " {{#select 'armor'}} leather {{or}} chainmail {{or}} plate {{/select}} ",
 "weapon": " {{select 'weapon' options=valid_weapons}} ",
 "class": " {{gen 'class'}} ",
 "mantra": " {{gen 'mantra' temperature=0.7}} ",
 "strength": {{gen 'strength' pattern='[0-9]+' stop=','}} ,
 "items": [ {{#geneach 'items' num_iterations=5 join=', '}} " {{gen 'this' temperature=0.7}} " {{/geneach}} ]
}```

In [29]:
results=character_maker(
    id="e1f491f7-7ab8-4dac-8c20-c92b5e7d883d",
    description="A quick and nimble fighter.",
    valid_weapons=valid_weapons, llm=llama
)

Stop program The following is a character profile for an RPG game in JSON format.
```json
{
 "id": " {{id}} ",
 "description": " {{description}} ",
 "name": " {{gen 'name'}} ",
 "age": {{gen 'age' pattern='[0-9]+' stop=','}} ,
 "armor": " {{#select 'armor'}} leather {{or}} chainmail {{or}} plate {{/select}} ",
 "weapon": " {{select 'weapon' options=valid_weapons}} ",
 "class": " {{gen 'class'}} ",
 "mantra": " {{gen 'mantra' temperature=0.7}} ",
 "strength": {{gen 'strength' pattern='[0-9]+' stop=','}} ,
 "items": [ {{#geneach 'items' num_iterations=5 join=', '}} " {{gen 'this' temperature=0.7}} " {{/geneach}} ]
}```

In [16]:
results.variables()

{'llm': <guidance.llms._transformers.Transformers at 0x7f569425fa00>,
 'logging': False,
 'id': 'e1f491f7-7ab8-4dac-8c20-c92b5e7d883d',
 'description': 'A quick and nimble fighter.',
 'valid_weapons': ['sword', 'axe', 'mace', 'spear', 'bow', 'crossbow'],
 '@raw_prefix': 'The following is a character profile for an RPG game in JSON format.\n```json\n{\n    "id": "{{!--GMARKER_START_variable_ref$&#123;&#123;id&#125;&#125;$--}}e1f491f7-7ab8-4dac-8c20-c92b5e7d883d{{!--GMARKER_END_variable_ref$$--}}",\n    "description": "{{!--GMARKER_START_variable_ref$&#123;&#123;description&#125;&#125;$--}}A quick and nimble fighter.{{!--GMARKER_END_variable_ref$$--}}",\n    "name": "{{!--GMARKER_START_gen$&#123;&#123;gen \'name\'&#125;&#125;$--}}Ryu",\n    "race": "Human",\n    "class": "Warrior",\n    "level": 10,\n    "stats": {\n        "strength": 18,\n        "dexterity": 16,\n        "constitution": 14,\n        "intelligence": 10,\n        "wisdom": 12,\n        "charisma": 14\n    },\n    "skill

In [6]:
article="""
The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]

In 1961, the US government put Jupiter nuclear missiles in Italy and Turkey. It had also trained a paramilitary force of Cuban exiles, which the CIA led in an attempt to invade Cuba and overthrow the Cuban government. Starting in November of that year, the US government engaged in a campaign of terrorism and sabotage in Cuba, referred to as the Cuban Project, which continued throughout the first half of the 1960s. The Soviet administration was concerned about a Cuban drift towards China, with which the Soviets had an increasingly fractious relationship. In response to these factors, Soviet First Secretary, Nikita Khrushchev, agreed with the Cuban Prime Minister, Fidel Castro, to place nuclear missiles on the island of Cuba to deter a future invasion. An agreement was reached during a secret meeting between Khrushchev and Castro in July 1962, and construction of a number of missile launch facilities started later that summer.

During the campaigning for the 1962 United States elections, the White House denied the charges for months and ignored the presence of Soviet missiles positioned approximately 90 mi (140 km) away from Florida. Later, the missile preparations were confirmed when a US Air Force U-2 spy plane produced clear photographic evidence of medium-range R-12 (NATO code name SS-4) and intermediate-range R-14 (NATO code name SS-5) ballistic missile facilities.

When this was reported to President John F. Kennedy, he then convened a meeting of the nine members of the National Security Council and five other key advisers, in a group that became known as the Executive Committee of the National Security Council (EXCOMM). During this meeting, President Kennedy was originally advised to carry out an air strike on Cuban soil in order to compromise Soviet missile supplies, followed by an invasion of the Cuban mainland. After careful consideration, President Kennedy chose a less aggressive course of action, in order to avoid a declaration of war. After consultation with EXCOMM, Kennedy ordered a naval "quarantine" on October 22 to prevent further missiles from reaching Cuba.[5] By using the term "quarantine", rather than "blockade" (an act of war by legal definition), the United States was able to avoid the implications of a state of war.[6] The US announced it would not permit offensive weapons to be delivered to Cuba and demanded that the weapons already in Cuba be dismantled and returned to the Soviet Union.

After several days of tense negotiations, an agreement was reached between Kennedy and Khrushchev: publicly, the Soviets would dismantle their offensive weapons in Cuba and return them to the Soviet Union, subject to United Nations verification, in exchange for a US public declaration and agreement to not invade Cuba again. Secretly, the United States agreed with the Soviets that it would dismantle all of the Jupiter MRBMs which had been deployed to Turkey against the Soviet Union. There has been debate on whether or not Italy was included in the agreement as well. While the Soviets dismantled their missiles, some Soviet bombers remained in Cuba, and the United States kept the naval quarantine in place until November 20, 1962.[6]

When all offensive missiles and the Ilyushin Il-28 light bombers had been withdrawn from Cuba, the blockade was formally ended on November 20. The negotiations between the United States and the Soviet Union pointed out the necessity of a quick, clear, and direct communication line between the two superpowers. As a result, the Moscow–Washington hotline was established. A series of agreements later reduced US–Soviet tensions for several years, until both parties eventually resumed expanding their nuclear arsenals.

The compromise embarrassed Khrushchev and the Soviet Union because the withdrawal of US missiles from Italy and Turkey was a secret deal between Kennedy and Khrushchev, and the Soviets were seen as retreating from circumstances that they had started. Khrushchev's fall from power two years later was in part because of the Soviet Politburo's embarrassment at both Khrushchev's eventual concessions to the US and his ineptitude in precipitating the crisis in the first place. According to Dobrynin, the top Soviet leadership took the Cuban outcome as "a blow to its prestige bordering on humiliation".[7][8]
"""
print(len(article)/4)

1278.75


In [7]:
questions="""

"""

answers="""
### Begin Answers
{
"""


In [20]:
valid_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]

# define the prompt
character_maker = guidance("""
Carefully read this article and then answer the list of questions.

### Start Article
The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]
### End Article

Now answer the following questions. Be concise and truthful. Format your responses as json values and nothing else. Include only the answers no other text. Only include information directly mentioned in the article.

### Begin Questions
Q. Who were the combatants in the crisis (countries)?
A. {{gen 'mantra' temperature=0.7}}
### End Questions
"""
)

,
    "Which countries were involved in any way in the crisis (countries)?": "{{gen 'a2' temperature=0.7 stop=',' max_tokens=20}}",
    "When did the crisis start (dates)?": "{{gen 'a3' temperature=0.7 stop=',' max_tokens=20}}",
}```

In [21]:
# generate a character
results = character_maker(
    article=article,
    llm=llama
)

Stop program 
Carefully read this article and then answer the list of questions.

### Start Article
The Cuban Missile Crisis, also known as the October Crisis (of 1962) (Spanish: Crisis de Octubre) in Cuba, the Caribbean Crisis (Russian: Карибский кризис, tr. Karibsky krizis, IPA: [kɐˈrʲipskʲɪj ˈkrʲizʲɪs]) in Russia, or the Missile Scare, was a 13-day (October 16 – October 29, 1962) confrontation between the United States and the Soviet Union, which escalated into an international crisis when American deployments of missiles in Italy and Turkey were matched by Soviet deployments of similar ballistic missiles in Cuba. Despite the short time frame, the Cuban Missile Crisis remains a defining moment in American national security and nuclear war preparation. The confrontation is often considered the closest the Cold War came to escalating into a full-scale conflict, nuclear war.[4]
### End Article

Now answer the following questions. Be concise and truthful. Format your responses as json values and nothing else. Include only the answers no other text. Only include information directly mentioned in the article.

### Begin Questions
Q. Who were the combatants in the crisis (countries)?
A. {{gen 'mantra' temperature=0.7}} 
### End Questions

KeyboardInterrupt: 

In [ ]:
results.variables()